In [1]:
import os
import cv2
import numpy as np
from google.colab import drive
import albumentations as A

# Mount Google Drive
drive.mount('/content/drive')

# Path dataset
path = "/content/drive/MyDrive/pmk_dataset/yolo/kaki/train"

Mounted at /content/drive


In [2]:
# Target jumlah foto per kelas
TARGET_PHOTOS = 30

# Definisi augmentasi (ringan untuk data medis)
def get_augmentation():
    return A.Compose([
        A.OneOf([
            A.Rotate(limit=15, p=0.8),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=3, p=0.3),
        ], p=1.0)
    ])

In [3]:
# Fungsi untuk augmentasi per kelas
def augment_class(class_folder_path, class_name, target_count):
    print(f"\n=== Memproses Kelas {class_name} ===")

    # Dapatkan semua file gambar
    image_files = [f for f in os.listdir(class_folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    original_count = len(image_files)
    needed_count = target_count - original_count

    print(f"Foto asli: {original_count}")
    print(f"Foto dibutuhkan: {needed_count}")

    if needed_count <= 0:
        print("Sudah cukup, tidak perlu augmentasi!")
        return

    # Siapkan augmentasi
    transform = get_augmentation()

    # Hitung berapa kali harus mengulang
    augmentations_per_image = needed_count // original_count
    extra_augmentations = needed_count % original_count

    print(f"Augmentasi per foto: {augmentations_per_image}")
    print(f"Augmentasi tambahan: {extra_augmentations}")

    counter = 1

    # Lakukan augmentasi
    for i, image_file in enumerate(image_files):
        # Baca gambar
        image_path = os.path.join(class_folder_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Nama dasar untuk file augmented
        base_name = os.path.splitext(image_file)[0]

        # Augmentasi reguler
        for j in range(augmentations_per_image):
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            # Simpan dengan nama yang dinomori
            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

        # Augmentasi tambahan (jika diperlukan)
        if i < extra_augmentations:
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

    print(f"Selesai! Total foto sekarang: {len(os.listdir(class_folder_path))}")

In [4]:
# Jalankan augmentasi untuk semua kelas
def main():
    # Install albumentations jika belum ada
    try:
        import albumentations
    except ImportError:
        print("Installing albumentations...")
        os.system("pip install albumentations")
        import albumentations as A

    print("=== MULAI AUGMENTASI DATA PMK ===")
    print(f"Target: {TARGET_PHOTOS} foto per kelas")

    # Loop untuk setiap kelas (0, 1, 2, 3)
    classes = ['0', '1', '2', '3']

    for class_name in classes:
        class_path = os.path.join(path, class_name)

        if os.path.exists(class_path):
            augment_class(class_path, class_name, TARGET_PHOTOS)
        else:
            print(f"Folder kelas {class_name} tidak ditemukan!")

    print("\n=== AUGMENTASI SELESAI ===")

    # Tampilkan summary
    print("\n=== SUMMARY ===")
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        if os.path.exists(class_path):
            total_files = len([f for f in os.listdir(class_path)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"Kelas {class_name}: {total_files} foto")

# Jalankan program
if __name__ == "__main__":
    main()

=== MULAI AUGMENTASI DATA PMK ===
Target: 30 foto per kelas

=== Memproses Kelas 0 ===
Foto asli: 20
Foto dibutuhkan: 10
Augmentasi per foto: 0
Augmentasi tambahan: 10


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: WhatsApp Image 2022-11-10 at 7.53.26 PM (1)_01.jpg
  Dibuat: IMG_20221105_105514_02.jpg
  Dibuat: IMG_20221105_105439_03.jpg
  Dibuat: IMG_20221105_105505_04.jpg
  Dibuat: IMG_20221105_105352_05.jpg
  Dibuat: IMG_20221105_105309_06.jpg
  Dibuat: istockphoto-1279979554-612x612_07.jpg
  Dibuat: IMG_20221105_105320_08.jpg
  Dibuat: IMG_20221105_105414_09.jpg
  Dibuat: IMG_20221105_105307_10.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 1 ===
Foto asli: 6
Foto dibutuhkan: 24
Augmentasi per foto: 4
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: kaki_17p_01.jpg
  Dibuat: kaki_17p_02.jpg
  Dibuat: kaki_17p_03.jpg
  Dibuat: kaki_17p_04.jpg
  Dibuat: kaki_7p_05.jpg
  Dibuat: kaki_7p_06.jpg
  Dibuat: kaki_7p_07.jpg
  Dibuat: kaki_7p_08.jpg
  Dibuat: kaki_16p_09.jpg
  Dibuat: kaki_16p_10.jpg
  Dibuat: kaki_16p_11.jpg
  Dibuat: kaki_16p_12.jpg
  Dibuat: kaki_1_13.jpg
  Dibuat: kaki_1_14.jpg
  Dibuat: kaki_1_15.jpg
  Dibuat: kaki_1_16.jpg
  Dibuat: kaki_4_17.jpg
  Dibuat: kaki_4_18.jpg
  Dibuat: kaki_4_19.jpg
  Dibuat: kaki_4_20.jpg
  Dibuat: download_21.jpg
  Dibuat: download_22.jpg
  Dibuat: download_23.jpg
  Dibuat: download_24.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 2 ===
Foto asli: 7
Foto dibutuhkan: 23
Augmentasi per foto: 3
Augmentasi tambahan: 2


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: kaki_9p_01.jpg
  Dibuat: kaki_9p_02.jpg
  Dibuat: kaki_9p_03.jpg
  Dibuat: kaki_9p_04.jpg
  Dibuat: kaki_6p_05.jpg
  Dibuat: kaki_6p_06.jpg
  Dibuat: kaki_6p_07.jpg
  Dibuat: kaki_6p_08.jpg
  Dibuat: kaki_15p_09.jpg
  Dibuat: kaki_15p_10.jpg
  Dibuat: kaki_15p_11.jpg
  Dibuat: kaki_2p_12.jpg
  Dibuat: kaki_2p_13.jpg
  Dibuat: kaki_2p_14.jpg
  Dibuat: kaki_14p_15.jpg
  Dibuat: kaki_14p_16.jpg
  Dibuat: kaki_14p_17.jpg
  Dibuat: kaki_8p_18.jpg
  Dibuat: kaki_8p_19.jpg
  Dibuat: kaki_8p_20.jpg
  Dibuat: kaki_1p_21.jpg
  Dibuat: kaki_1p_22.jpg
  Dibuat: kaki_1p_23.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 3 ===
Foto asli: 2
Foto dibutuhkan: 28
Augmentasi per foto: 14
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: kaki_3_01.jpg
  Dibuat: kaki_3_02.jpg
  Dibuat: kaki_3_03.jpg
  Dibuat: kaki_3_04.jpg
  Dibuat: kaki_3_05.jpg
  Dibuat: kaki_3_06.jpg
  Dibuat: kaki_3_07.jpg
  Dibuat: kaki_3_08.jpg
  Dibuat: kaki_3_09.jpg
  Dibuat: kaki_3_10.jpg
  Dibuat: kaki_3_11.jpg
  Dibuat: kaki_3_12.jpg
  Dibuat: kaki_3_13.jpg
  Dibuat: kaki_3_14.jpg
  Dibuat: kaki_5p_15.jpg
  Dibuat: kaki_5p_16.jpg
  Dibuat: kaki_5p_17.jpg
  Dibuat: kaki_5p_18.jpg
  Dibuat: kaki_5p_19.jpg
  Dibuat: kaki_5p_20.jpg
  Dibuat: kaki_5p_21.jpg
  Dibuat: kaki_5p_22.jpg
  Dibuat: kaki_5p_23.jpg
  Dibuat: kaki_5p_24.jpg
  Dibuat: kaki_5p_25.jpg
  Dibuat: kaki_5p_26.jpg
  Dibuat: kaki_5p_27.jpg
  Dibuat: kaki_5p_28.jpg
Selesai! Total foto sekarang: 30

=== AUGMENTASI SELESAI ===

=== SUMMARY ===
Kelas 0: 30 foto
Kelas 1: 30 foto
Kelas 2: 30 foto
Kelas 3: 30 foto
